In [3]:
import os
# check if the folder exists
directory = "data/online"

if not os.path.exists(directory):
    link = "https://www.dropbox.com/scl/fi/nkupgd6mfrxryefdoxxwn/20_30.zip?rlkey=dpwjhieorllwvavfh3le622t8&st=koo7vhx8&dl=1"
    import requests
    import zipfile
    import io

    r = requests.get(link)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(directory)



In [ ]:

# open all json files and parse them
import json
import os

directory = "data/online/20_30"
data = []
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        with open(os.path.join(directory, filename)) as f:
            data.append(json.load(f))

# map all data in the grid
# they all the combination of policies versus a different policy
# I want to map them in a grid showing the win rate

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# get all the policies
policies = set()
for d in data:
    policies.add(d["policies_names"][0])
policies = list(policies)
policies.sort()
index_policy = {k: i for i, k in enumerate(policies)}
# create the grid
grid = np.zeros((len(policies), len(policies)))

for d in data:
    x, y = index_policy[d["policies_names"][0]], index_policy[d["policies_names"][1]]
    # calculate the win rate
    wins = 0
    for battle in d["battles"]:
        if battle["result"] == 0:
            wins += 1
    grid[x, y] = wins / len(d["battles"])
    grid[y, x] = 1 - grid[x, y]

# Define the custom colormap
colors = ['#b10407', '#ffff00', '#058618']  # R -> Y -> G
n_bins = 100  # Discretize the interpolation into bins
cmap_name = 'r_y_g'
cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins)
n = len(policies)
fig, ax = plt.subplots()
cax = ax.matshow(grid, cmap=cm)
for i in range(n):
    for j in range(n):
        ax.text(j, i, f"{grid[i][j]:.2f}", ha='center', va='center', color='black')
plt.xticks(range(n), policies, rotation=45, ha='left')
plt.yticks(range(n), policies)

